 ### Using Twitter to Build your own Dataset

[Introduction to tweepy](https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25)

In [18]:
import tweepy
import datetime
import pandas as pd
import time

In [19]:
mp = pd.read_csv('all_uk_mps.csv')

mp.head()

,Name,Screen Name,Constituency,Party,Followers
0,Adam Afriyie,AdamAfriyie,Windsor,Conservative,"13,520"
1,Alan Duncan,AlanDuncanMP,Rutland and Melton,Conservative,"23,984"
2,Alan Mak,AlanMakMP,Havant,Conservative,"7,844"
3,Alberto Costa,AlbertoCostaMP,South Leicestershire,Conservative,"8,358"
4,Alec Shelbrooke,AlecShelbrooke,Elmet and Rothwell,Conservative,"14,473"


In [20]:
#create list of screen names to call
screen_names = mp['Screen Name']

In [4]:
#Set up miner class
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
    'consumer_key':        '-------YOUR KEY HERE-------',
    'consumer_secret':     '-------YOUR KEY HERE-------',
    'access_token_key':    '-------YOUR KEY HERE-------',
    'access_token_secret': '-------YOUR KEY HERE-------'
}
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dril", #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data        

In [21]:
### SAME AS ABOVE BUT WITH MY API KEYS IN FOR TESTING -

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
    'consumer_key':        '9PI5X4qo8t9RH4qMNG0V8LIQ2',
    'consumer_secret':     'VepEHjLYiTAo4iiMzeZd0mNv64Y5xqCWcnMvThpW38dCG3EjgF',
    'access_token_key':    '545158426-RygXZ6vYeFO4Nn0BIOcFutjH6OC5WeHfQzBBemQy',
    'access_token_secret': 'wO77FsvPX7VF4JX0B6RVzax7SMLRsd48w6GJdG6v7QpNH'
}
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dril", #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data    

In [41]:
def twitter_call(screen_names, result_limit=20, max_pages=1, sleep=0, sleep_counter=40):
    """screen_names: list of strings - should not include the @ precedding username
       result_limit: (0-200)
       max_pages: (0-17)
       sleep: (0-900)
       sleep_activate: 
       
       max_pages * result_limit =  number of tweets collected for each handle
       Cap on tweets collected = 3200 (result_limit=200, max_pages=17)
       sleep timer should be set to 900 when collecting max tweets for each username i.e. 15 mins, at which
       point limits are refreshed 
       sleep counter sets number of usernames called before sleep timer kicks in, 40 works for max tweets, 
       can be increased if tweets collected per user is reduced (proportional adjustment should work)
       """
    
    
    counter=0
    twitter_dict={}
    
    
    print(len(screen_names), ' accounts in list')
    
    for name in screen_names:
        try:
            twitter_dict[name]=[]
            twitter_dict[name].append(miner.mine_user_tweets(user=name, max_pages=max_pages))
            counter = counter +1
            time.sleep(.1) #makes calls more reliable
            if counter%sleep_counter==0:
                print(counter, ' completed')
                time.sleep(sleep) #15 minute sleep time
                
        #if name invalid print name and remove key
        except:
            print(name, ' not added (invalid, locked or api limit hit)')
            twitter_dict.pop(name)
    
    tweet_df = pd.concat([pd.DataFrame(twitter_dict[i][0]) for i in twitter_dict])
    tweet_df.reset_index(inplace=True, drop=True)
    
    return tweet_df

In [42]:
tweets = twitter_call(screen_names[:3]) #make call on first mps, 2 tweets each

3  accounts in list


In [47]:
tweets.head()

,created_at,favourite_count,hashtags,location,mined_at,name,quote_screen_name,quote_text,retweet_count,retweet_text,screen_name,source_device,status_count,text,tweet_id
0,2019-07-06 17:27:09,19,[],None,2019-07-11 16:03:47.643887,Adam Afriyie,None,None,0,None,AdamAfriyie,Twitter for iPhone,1773,Thanks to Nigel &amp; Suzanne Kneighton for ho...,1147557609119129601
1,2019-07-03 14:10:10,3,[],None,2019-07-11 16:03:47.643927,Adam Afriyie,None,None,1,None,AdamAfriyie,Twitter for iPhone,1773,Delighted to support the British Bangladeshi C...,1146420874976878593
2,2019-07-11 14:11:03,6,[],None,2019-07-11 16:03:47.972857,Sir Alan Duncan MP,None,None,0,None,AlanDuncanMP,Twitter Web Client,1075,A warm welcome to Uzbekistan Foreign Minister ...,1149320200665350149
3,2019-07-10 19:14:26,25,[],None,2019-07-11 16:03:47.972872,Sir Alan Duncan MP,None,None,4,None,AlanDuncanMP,Twitter for iPhone,1075,Delighted to open the batting at @foreignoffic...,1149034160129884160
4,2019-07-11 13:13:53,0,[],None,2019-07-11 16:03:48.292269,Alan Mak MP,None,None,2,@the_tpa and you can read @AlanMakMP's report ...,AlanMakMP,Twitter for Android,4763,RT @CPSThinkTank: @the_tpa and you can read @A...,1149305814236192768


In [49]:
#Try your own call and add it to the tweet library
tweets = twitter_call(screen_names =   , result_limit=  , max_pages=  , sleep=0, sleep_counter=40)

1  accounts in list


In [51]:
tweets.to_csv('descriptive_name.csv')

## Sentiment analysis

[5 Heroic NLP Libraries](https://elitedatascience.com/python-nlp-libraries)
